# Tables in main text

## Create dataframe - Stage Races

In [129]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy
import re
from scipy import stats
import statsmodels.formula.api as sm
from stargazer.stargazer import Stargazer, LineLocation
from scipy.stats import chi2_contingency

# Load and clean the 'stage_races_nf15.xlsx' dataset
stage = pd.read_excel('stage_races_nf15.xlsx')
stage = stage.drop(stage.columns[0], axis=1)  # Drop the first unnamed column
stage = stage.drop(columns=['Team_Score'])  # Drop the 'Team_Score' column
stage = stage.drop_duplicates()  # Remove duplicate rows

# Load and clean the 'itts.xlsx' dataset (individual time trials)
itt = pd.read_excel('itts.xlsx')
itt = itt.drop(itt.columns[0], axis=1)  # Drop the first unnamed column
itt = itt.drop_duplicates()  # Remove duplicate rows

# Find common rows between 'stage' and 'itt' based on 'Race' and 'Stage' columns
common_rows = stage.merge(itt, on=['Race', 'Stage'])

# Remove the common rows from the 'stage' dataset
stage_noitt = stage[~stage.set_index(['Race', 'Stage']).index.isin(itt.set_index(['Race', 'Stage']).index)]

# Reset the index for the resulting 'stage_noitt' dataframe
stage_noitt.reset_index(drop=True, inplace=True)

# Update 'stage' to reflect the dataset without individual time trial (ITT) stages
stage = stage_noitt


In [130]:
# Initialize columns related to teammates, clusters, and rider roles
stage['Teammates'] = 0
stage['Cluster_size_teams'] = 1
stage['Cluster_size_teams_hyp'] = 1
stage['Star'] = 0
stage['not_a_Star'] = 0
stage['Star_other'] = 0 
stage['Star_other_team'] = 0
stage['Star_my_team'] = 0
stage['Star_of_Cluster'] = 0
stage['Star_other_in_Cluster'] = 0
stage['Star_other_team_in_Cluster'] = 0
stage['Star_my_team_in_Cluster'] = 0
stage['Helper_in_Cluster'] = 0
stage['Captain_in_Cluster'] = 0
stage['Helper_hyp_in_Cluster'] = 0
stage['Captain_hyp_in_Cluster'] = 0
stage['Star_in_Cluster1'] = 0
stage['Star_in_Cluster2'] = 0
stage['Winner_is_Star'] = 0
stage['Teammates_behind'] = 0
stage['eliminate'] = 0
stage['Teammates_behind_hyp'] = 0
stage['Star_Teammate_behind'] = 0
stage['Teammates_front'] = 0
stage['Teammates_front_hyp'] = 0
stage['Win'] = 0
stage['Cluster_size'] = 1
stage['Cluster'] = 1

# Mark winners and extract year from stage info
stage.loc[stage['Place'] == 1, 'Win'] = 1
stage['Year'] = stage['Stage'].str.split(':', expand=True)[0].astype(int)

### Stars

In [131]:
# Loop through years to create dummy variables for each year and identify stars
for i in range(1981, 2024):
    stage[f'Dummy_{i}'] = 0
    stage.loc[stage['Year'] == i, f'Dummy_{i}'] = 1
    threshold_up = stage.loc[stage['Year'] == i, 'Score'].quantile(0.80)
    threshold_down = stage.loc[stage['Year'] == i, 'Score'].quantile(0.20)
    
    stage.loc[stage['Year'] == i, 'Star'] = (stage.loc[stage['Year'] == i, 'Score'] >= threshold_up).astype(int)
    stage.loc[stage['Year'] == i, 'not_a_Star'] = (stage.loc[stage['Year'] == i, 'Score'] < threshold_up).astype(int)

# Clean up race names and create dummy variables for each race
for race in stage['Race'].unique():
    cleaned_race = race.replace('/', '').replace('-', '_')
    stage.loc[stage['Race'] == race, 'Race'] = cleaned_race
    stage[f'Dummy_{cleaned_race}'] = (stage['Race'] == cleaned_race).astype(int)

# Create dummy variables for each stage type
for stagetype in stage['Stagetype'].unique():
    stage[f'Dummy_stagetype_{stagetype}'] = (stage['Stagetype'] == stagetype).astype(int)

# Create additional composite identifiers
stage['Race_Stage'] = stage['Race'] + stage['Stage'].astype(str)  # E.g., "2003: 2"
stage['Race_Year'] = stage['Race'] + stage['Year'].astype(str)  # E.g., "2003: 2"

# Assign hypothetical teams (randomly assigning teams 1 to 22)
stage['hyp_team'] = np.random.randint(1, 23, size=len(stage))


### Groups

In [132]:
# Iterate through unique race stages to define clusters and teammate roles
for group_name in stage['Race_Stage'].unique():
    group_data = stage[stage['Race_Stage'] == group_name]
    
    # Define clusters based on time gaps
    for i in range(1, len(group_data)):
        gap_difference = group_data.iloc[i]['Gap'] - group_data.iloc[i - 1]['Gap']
        if gap_difference > 4:  # 5+ seconds gap creates a new cluster
            stage.loc[group_data.index[i], 'Cluster'] = stage.loc[group_data.index[i - 1], 'Cluster'] + 1
            stage.loc[group_data.index[i], 'Gap_front'] = gap_difference
        else:
            stage.loc[group_data.index[i], 'Gap_front'] = 0
            stage.loc[group_data.index[i], 'Cluster'] = stage.loc[group_data.index[i - 1], 'Cluster']
    
    # Update cluster sizes and teammate information
    for i in range(len(group_data)):
        for j in range(i + 1, len(group_data)):
            same_cluster = stage.loc[group_data.index[i], 'Cluster'] == stage.loc[group_data.index[j], 'Cluster']
            same_team = stage.loc[group_data.index[i], 'Team'] == stage.loc[group_data.index[j], 'Team']
            if same_cluster:
                stage.loc[group_data.index[i], 'Cluster_size'] += 1
                stage.loc[group_data.index[j], 'Cluster_size'] += 1
                if same_team:
                    # Mark teammates in same cluster
                    stage.loc[group_data.index[i], 'Helper_in_Cluster'] = 1
                    stage.loc[group_data.index[j], 'Captain_in_Cluster'] = 1
                    stage.loc[group_data.index[i], 'Teammates'] = 1
                    stage.loc[group_data.index[j], 'Teammates'] = 1
                else:
                    # Non-teammates in the same cluster
                    pass
                
            # Mark teammates in neighboring clusters
            if same_team and stage.loc[group_data.index[i], 'Cluster'] + 1 == stage.loc[group_data.index[j], 'Cluster']:
                stage.loc[group_data.index[i], 'Teammates_behind'] = 1
                stage.loc[group_data.index[j], 'Teammates_front'] = 1
            
            if same_team and stage.loc[group_data.index[i], 'Cluster'] + 2 == stage.loc[group_data.index[j], 'Cluster']:
                stage.loc[group_data.index[j], 'Teammates_front'] = 1
            
            # Hypothetical teammates in same cluster
            if same_cluster and stage.loc[group_data.index[i], 'hyp_team'] == stage.loc[group_data.index[j], 'hyp_team']:
                stage.loc[group_data.index[i], 'Helper_hyp_in_Cluster'] = 1
                stage.loc[group_data.index[j], 'Captain_hyp_in_Cluster'] = 1
                
            # Hypothetical teammates in neighboring clusters
            if stage.loc[group_data.index[i], 'Cluster'] + 1 == stage.loc[group_data.index[j], 'Cluster'] and stage.loc[group_data.index[i], 'hyp_team'] == stage.loc[group_data.index[j], 'hyp_team']:
                stage.loc[group_data.index[i], 'Teammates_behind_hyp'] = 1
                stage.loc[group_data.index[j], 'Teammates_front_hyp'] = 1


### Remaining code

In [133]:
# Calculate the number of unique teams per cluster
stage['Cluster_size_teams'] = stage.groupby(['Race_Stage', 'Cluster'])['Team'].transform('nunique')

# Create dummy variables for clusters
for s in stage['Cluster'].unique():
    stage.loc[stage['Cluster'] == 1, 'Dummy_Cluster_1'] = 1
    stage.loc[stage['Cluster'] != 1, 'Dummy_Cluster_1'] = 0

# Identify the winners of each race stage and merge their cluster size information
winners = stage[stage['Place'] == 1][['Race_Stage', 'Cluster_size', 'Cluster_size_teams']]
stage = pd.merge(stage, winners, on='Race_Stage', suffixes=('', '_winner'), how='left')

# Identify the cluster size for the second cluster and merge with the main dataset
second = stage[stage['Cluster'] == 2][['Race_Stage', 'Cluster_size', 'Cluster_size_teams']]
stage = pd.merge(stage, second, on='Race_Stage', suffixes=('', '_second'), how='left')

# Identify the cluster size for the third cluster and merge with the main dataset
third = stage[stage['Cluster'] == 3][['Race_Stage', 'Cluster_size', 'Cluster_size_teams']]
stage = pd.merge(stage, third, on='Race_Stage', suffixes=('', '_third'), how='left')

# Filter and mark races for elimination based on conditions
for s in stage['Race_Stage'].unique():
    group_data = stage.loc[stage['Race_Stage'] == s]
    
    # Mark races for elimination if a rider in cluster 1, 2, or 3 places 15th
    for i in range(len(group_data)):
        if (stage.loc[group_data.index[i], 'Place'] == 15) and (stage.loc[group_data.index[i], 'Cluster'] in [1, 2, 3]):
            stage.loc[group_data.index[i], 'eliminate'] = 1
        
        # Eliminate races where both the first and second clusters have only one team each
        if (stage.loc[group_data.index[i], 'Cluster_size_teams_winner'] == 1) and (stage.loc[group_data.index[i], 'Cluster_size_teams_second'] == 1):
            stage.loc[group_data.index[i], 'eliminate'] = 1

# Identify 'Race_Stage' values that should be eliminated
eliminate_race_stages = stage.loc[stage['eliminate'] == 1, 'Race_Stage'].unique()

# Apply elimination to all rows with the identified 'Race_Stage' values
stage.loc[stage['Race_Stage'].isin(eliminate_race_stages), 'eliminate'] = 1

# Remove rows marked for elimination
stage = stage[stage['eliminate'] != 1].copy()

# Drop duplicates and lay focus on first three clusters
stage = stage.drop_duplicates()
stage_filtered = stage[stage['Cluster'] <= 3].copy()

# Reset the index of the filtered DataFrame
stage_filtered.reset_index(drop=True, inplace=True)

# Update the main 'stage' DataFrame with the filtered data
stage = stage_filtered

# Print the number of stages used after filtering
print('We use a total of', len(stage['Race_Stage'].unique()), 'stages of stage races.')

# Step 1: Identify if there are other 'Stars' in the same cluster
# Create a unique 'Cluster_id' for each combination of Race_Stage and Cluster
stage['Cluster_id'] = stage['Race_Stage'] + stage['Cluster'].astype(str)

# Group by 'Cluster_id' and count the number of 'Stars' in each cluster
grouped_data = stage.groupby('Cluster_id')['Star']
sum_star = grouped_data.transform('sum')

# Mark if there is another 'Star' in the cluster (either from the same or different team)
stage['Star_other_in_Cluster'] = (((sum_star >= 2) & (stage['Star'] == 1)) | 
                                  ((sum_star >= 1) & (stage['Star'] != 1))).astype(int)

# Step 2: Identify if there is another 'Star' from a different team or the same team in the cluster
for cluster_id in stage['Cluster_id'].unique():
    group_data = stage.loc[stage['Cluster_id'] == cluster_id]
    
    # Loop through each rider in the cluster and check for 'Star' teammates or 'Stars' from other teams
    for i in range(len(group_data)):
        for j in range(len(group_data)):
            # Other 'Star' from a different team
            if (stage.loc[group_data.index[i], 'Team'] != stage.loc[group_data.index[j], 'Team']) and (stage.loc[group_data.index[j], 'Star'] == 1):
                stage.loc[group_data.index[i], 'Star_other_team_in_Cluster'] = 1
            
            # Other 'Star' from the same team (not the current rider)
            if (stage.loc[group_data.index[i], 'Team'] == stage.loc[group_data.index[j], 'Team']) and (stage.loc[group_data.index[j], 'Star'] == 1) and (i != j):
                stage.loc[group_data.index[i], 'Star_my_team_in_Cluster'] = 1

# Drop the 'Cluster_id' column as it is no longer needed
stage.drop('Cluster_id', axis=1, inplace=True)

# Loop through each unique 'Race_Stage'
for race_stage in stage['Race_Stage'].unique():
    # Filter the data for the current race stage
    group_data = stage.loc[stage['Race_Stage'] == race_stage]
    
    # Loop through each rider in the current race stage
    for i in range(len(group_data)):
        # Check if there is another 'Star' in clusters 1 or 2
        stage.loc[group_data.index[i], 'Star_other'] = (np.sum(group_data[(group_data['Cluster'] == 1) | (group_data['Cluster'] == 2)]['Star']) > stage.loc[group_data.index[i], 'Star']).astype(int)
        
        # Check if there is a 'Star' in Cluster 1
        stage.loc[group_data.index[i], 'Star_in_Cluster1'] = (np.sum(group_data[group_data['Cluster'] == 1]['Star']) > 0).astype(int)
        
        # Check if there is a 'Star' in Cluster 2
        stage.loc[group_data.index[i], 'Star_in_Cluster2'] = (np.sum(group_data[group_data['Cluster'] == 2]['Star']) > 0).astype(int)
        
        # Check if the winner is a 'Star'
        stage.loc[group_data.index[i], 'Winner_is_Star'] = (np.sum(group_data[group_data['Win'] == 1]['Star']) > 0).astype(int)
        
        # Calculate the maximum gap between Cluster 1 and Cluster 2
        stage.loc[group_data.index[i], 'Gap_Cluster12'] = group_data[group_data['Cluster'] == 2]['Gap_front'].max()
        
        # Calculate the maximum gap between Cluster 2 and Cluster 3
        stage.loc[group_data.index[i], 'Gap_Cluster23'] = group_data[group_data['Cluster'] == 3]['Gap_front'].max()
        
        # Check if there is a 'Helper' in Cluster 2
        stage.loc[group_data.index[i], 'Helper_in_Cluster2'] = (np.sum(group_data[group_data['Cluster'] == 2]['Helper_in_Cluster']) > 0).astype(int)
        
        # Check if the winner is part of a 'Satellite' group (teammates behind)
        stage.loc[group_data.index[i], 'Winner_is_Satellite'] = (np.sum(group_data[group_data['Win'] == 1]['Teammates_behind']) > 0).astype(int)
        
        # Calculate the standard deviation of scores in Cluster 2
        stage.loc[group_data.index[i], 'Cluster2_std'] = group_data[group_data['Cluster'] == 2]['Score'].std()
       
        # Check if there is a 'Star' from another team in clusters 1 or 2
        if (stage.loc[group_data.index[i], 'Team'] != stage.loc[group_data.index[j], 'Team']) and ((stage.loc[group_data.index[j], 'Cluster'] == 1) | (stage.loc[group_data.index[j], 'Cluster'] == 2)) and (stage.loc[group_data.index[j], 'Star'] == 1) and (i != j):
            stage.loc[group_data.index[i], 'Star_other_team'] = 1
            
            # Check if there is a 'Star' from the same team in clusters 1 or 2
        if (stage.loc[group_data.index[i], 'Team'] == stage.loc[group_data.index[j], 'Team']) and ((stage.loc[group_data.index[j], 'Cluster'] == 1) | (stage.loc[group_data.index[j], 'Cluster'] == 2)) and (stage.loc[group_data.index[j], 'Star'] == 1) and (i != j):
            stage.loc[group_data.index[i], 'Star_my_team'] = 1

# Create variables indicating the absence of stars within the rider's team, other teams, and the cluster
stage['no_Star_my_team_in_Cluster'] = 1 - stage['Star_my_team_in_Cluster']
stage['no_Star_other_team_in_Cluster'] = 1 - stage['Star_other_team_in_Cluster']
stage['no_Star_other_team'] = 1 - stage['Star_other_team']
stage['no_Star'] = 1 - stage['Star']

# Create a variable indicating if there is a better rider in the cluster
# (i.e., dummy equal to 1 if the rider is not a Star but a Star exists in the cluster)
stage['better_rider_in_Cluster'] = stage.apply(lambda row: 1 if row['Star_other_team_in_Cluster'] == 1 and row['Star'] == 0 else 0, axis=1)

# Create a variable indicating if there is a better rider nearby (in the entire group)
stage['better_rider_around'] = stage.apply(lambda row: 1 if row['Star_other_team'] == 1 and row['Star'] == 0 else 0, axis=1)

# Identify solo wins (i.e., Cluster size for the winner equals 1)
stage['Solo_Win'] = (stage['Cluster_size_winner'] == 1).astype(int)

# Create dummy variables for the existence of helpers and gap sizes
stage['Helper_in_Cluster_exists'] = (stage['Cluster_size'] > stage['Cluster_size_teams']).astype(int)
stage['Gap_12_larger1'] = (stage['Gap_Cluster12'] >= 60).astype(int)  # Gap between Cluster 1 and Cluster 2
stage['Gap_23_larger1'] = (stage['Gap_Cluster23'] >= 60).astype(int)  # Gap between Cluster 2 and Cluster 3

# Identify if the standard deviation in Cluster 2 is larger than the mean standard deviation
stage['Cluster2_std_large'] = (stage['Cluster2_std'] >= stage['Cluster2_std'].mean()).astype(int)

# Remove duplicate rows
stage = stage.drop_duplicates()

# Filter for captains only (no teammates in front and no captains in the cluster)
stage_c = stage[(stage['Teammates_front'] == 0) & (stage['Captain_in_Cluster'] == 0)]

# Further filter captains to only include years after 1980 (since we don't have scores before 1981)
stage_c = stage_c[stage_c['Year'].astype(int) > 1980]

We use a total of 729 stages of stage races.


## Main Tables 4-7

In [134]:
# Table 4: Asymmetry in losing versus winning groups

# Step 1: Filter out races where the Solo winner has a helper in Cluster 1
df2 = stage_c[~stage_c.Race_Stage.isin(
    stage_c[(stage_c['Cluster_size'] >= 2) & 
            (stage_c['Cluster'] == 1) & 
            (stage_c['Cluster_size_teams'] == 1)].Race_Stage)]

# Step 2: Exclude races where the second place has a gap less than 10 seconds
df1 = df2[~df2.Race_Stage.isin(
    df2[(df2['Place'] == 2) & 
        (df2['Gap'] < 10)].Race_Stage)]

# Step 3: Select Stage 1 for Cluster 2 where cluster size is between 3 and 6
stage1 = df1[(df1['Cluster_size_teams_winner'] == 1) & 
             (df1['Cluster'] == 2) & 
             (df1['Cluster_size'] >= 3) & 
             (df1['Cluster_size'] <= 6)].copy()  # Use .copy() to avoid SettingWithCopyWarning

# Solo wins in Cluster 2
stage1['Cluster2_Solo'] = 1  # Solo win in Cluster 2
stage1['Cluster1_noSolo'] = 0  # Not a solo win in Cluster 1
stage1['Star_in_Cluster_exists'] = stage1['Star_in_Cluster2']  # Existence of Star in Cluster 2
stage1['Helper_in_Cluster_exists'] = (stage1['Cluster_size'] > stage1['Cluster_size_teams']).astype(int)  # Helper exists in Cluster 2
stage1['Solo_is_Satellite'] = 0  # No satellite win for Cluster 2
stage1['Star_has_Helper'] = 0  # Initialize Star_has_Helper as 0
stage1.loc[(stage1['Star'] + stage1['Helper_in_Cluster']) > 1, 'Star_has_Helper'] = 1
stage1['Star_w_Helper_exists'] = stage1.groupby('Race_Stage')['Star_has_Helper'].transform('max')  # Check if any Star has helper in the race

# Step 4: Select Stage 2 for Cluster 1 where cluster size is between 3 and 6
stage2 = df2[(df2['Cluster'] == 1) & 
             (df2['Cluster_size'] >= 3) & 
             (df2['Cluster_size'] <= 6)].copy()  # Use .copy() to avoid SettingWithCopyWarning

# Add new variables for stage2
stage2['Cluster2_Solo'] = 0  # Not a solo win in Cluster 2
stage2['Cluster1_noSolo'] = 1  # Solo win in Cluster 1
stage2['Star_in_Cluster_exists'] = stage2['Star_in_Cluster1']  # Existence of Star in Cluster 1
stage2['Helper_in_Cluster_exists'] = (stage2['Cluster_size'] > stage2['Cluster_size_teams']).astype(int)  # Helper exists in Cluster 1
stage2['Solo_is_Satellite'] = stage2['Winner_is_Satellite']  # Satellite win in Cluster 1
stage2['Star_has_Helper'] = 0  # Initialize Star_has_Helper as 0
stage2.loc[(stage2['Star'] + stage2['Helper_in_Cluster']) > 1, 'Star_has_Helper'] = 1
stage2['Star_w_Helper_exists'] = stage2.groupby('Race_Stage')['Star_has_Helper'].transform('max')  # Check if any Star has helper in the race

# Concatenate the two dataframes and remove duplicates
df_stage = pd.concat([stage1, stage2]).drop_duplicates('Race_Stage')

# Set heterogeneity indicator
df_stage['heterog'] = 0
df_stage.loc[(df_stage['Star_in_Cluster_exists'] + df_stage['Helper_in_Cluster_exists']) >= 1, 'heterog'] = 1

# Step 5: Combine stage1 and stage2
df_stage = pd.concat([stage1, stage2])

# Remove duplicate 'Race_Stage' entries
df_stage = df_stage.drop_duplicates('Race_Stage')

# Step 6: Create heterogeneity variable 'heterog' and drop mountain finishes
df_stage['heterog'] = 0
df_stage.loc[(df_stage['Star_in_Cluster_exists'] + df_stage['Helper_in_Cluster_exists']) >= 1, 'heterog'] = 1
df_stage = df_stage[df_stage['Dummy_stagetype_5'] == 0]  # Drop mountain finishes

# Print the cells of Table 4:
# Calculate and print the mean of Helper_in_Cluster_exists for Cluster2_Solo and Cluster1_noSolo
print(df_stage[df_stage['Cluster2_Solo'] == 1]['Helper_in_Cluster_exists'].mean())
print(df_stage[df_stage['Cluster1_noSolo'] == 1]['Helper_in_Cluster_exists'].mean())

# Calculate and print the mean of Star_in_Cluster_exists for Cluster2_Solo and Cluster1_noSolo
print(df_stage[df_stage['Cluster2_Solo'] == 1]['Star_in_Cluster_exists'].mean())
print(df_stage[df_stage['Cluster1_noSolo'] == 1]['Star_in_Cluster_exists'].mean())


0.14285714285714285
0.232
0.5238095238095238
0.616


In [135]:
# Table 5: Linear Probability Model: Being part of a winning Group (with 3 to 6 riders)

# LHS: versus Group behind Solo winner
resultNoSolo = sm.ols('Cluster1_noSolo ~ Star_in_Cluster_exists + Helper_in_Cluster_exists + Cluster_size_teams + Dummy_stagetype_1 + Dummy_stagetype_2 + Dummy_stagetype_3 + Dummy_stagetype_4', 
                                   data=df_stage).fit()

print(resultNoSolo.summary())

# RHS: versus riders not finishing as Group

# Step 1: Filter races where cluster size 1 and 2 are not too large
df = stage_c[(stage_c['Cluster_size_teams_winner'] < 3) & (stage_c['Cluster_size_teams_second'] <= 4)]

# Step 2: Only keep riders that are not too far away (Gap < 30)
df2 = df[df['Gap'] < 30]

# Step 3: Exclude races where Cluster 2 is too far away (Gap >= 40)
stage1 = df2[~df2.Race_Stage.isin(df2[(df2['Cluster'] == 2) & (df2['Gap'] >= 40)].Race_Stage)].copy()

# Step 4: Create group size variables for stage1
stage1['Group_size_teams'] = stage1.groupby('Race_Stage')['Team'].transform('nunique')
stage1['Group_size'] = stage1.groupby('Race_Stage')['Rider'].transform('nunique')

# Step 5: Add group and helper/star variables for stage1
stage1['Group_together'] = 0
stage1['Star_in_Cluster_exists'] = stage1.groupby('Race_Stage')['Star'].transform('max').astype(int)
stage1['Helper_in_Cluster_exists'] = (stage1['Group_size'] > stage1['Group_size_teams']).astype(int)

# Step 6: Prepare stage2 with no solo riders (modifying .loc to avoid SettingWithCopyWarning)
stage2 = stage_c[(stage_c['Cluster'] == 1) & (stage_c['Cluster_size'] >= 3) & (stage_c['Cluster_size'] <= 6)].copy()
stage2.loc[:, 'Group_together'] = 1
stage2.loc[:, 'Star_in_Cluster_exists'] = stage2['Star_in_Cluster1']
stage2.loc[:, 'Helper_in_Cluster_exists'] = (stage2['Cluster_size'] > stage2['Cluster_size_teams']).astype(int)
stage2.loc[:, 'Group_size_teams'] = stage2['Cluster_size_teams']
stage2.loc[:, 'Group_size'] = stage2['Cluster_size']

# Step 7: Combine stage1 and stage2 into a single DataFrame
df_stage = pd.concat([stage1, stage2]).drop_duplicates('Race_Stage')

# Step 8: Create a heterogeneity variable (heterog) indicating presence of star/helper in cluster
df_stage['heterog'] = 0
df_stage.loc[(df_stage['Star_in_Cluster_exists'] + df_stage['Helper_in_Cluster_exists']) >= 1, 'heterog'] = 1

# Step 9: Exclude mountain finishes (Dummy_stagetype_5 == 0)
df_stage = df_stage[df_stage['Dummy_stagetype_5'] == 0]

# Step 10: Fit an OLS model to predict 'Group_together' based on various variables
resultTog = sm.ols('Group_together ~ Star_in_Cluster_exists + Helper_in_Cluster_exists + Group_size_teams + Dummy_stagetype_1 + Dummy_stagetype_2 + Dummy_stagetype_3 + Dummy_stagetype_4', 
                                data=df_stage).fit()

# Print the summary of the OLS regression results
print(resultTog.summary())


                            OLS Regression Results                            
Dep. Variable:        Cluster1_noSolo   R-squared:                       0.019
Model:                            OLS   Adj. R-squared:                 -0.015
Method:                 Least Squares   F-statistic:                    0.5703
Date:                Tue, 15 Oct 2024   Prob (F-statistic):              0.780
Time:                        17:46:05   Log-Likelihood:                -145.53
No. Observations:                 209   AIC:                             307.1
Df Residuals:                     201   BIC:                             333.8
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept               

In [136]:
# Table 6: Linear Probability Model: Finishing in Group 1

# LHS: G1 if in G1/G2
resultS12 = sm.ols(formula='Dummy_Cluster_1 ~ better_rider_around + Teammates_behind + Gap_12_larger1 + Gap_23_larger1 + Cluster_size_teams_winner + Cluster_size_teams_second + Cluster_size_teams_third + Dummy_1982 + Dummy_1983 + Dummy_1985 + Dummy_1986 + Dummy_1987 + Dummy_1988 + Dummy_1989 + Dummy_1990 + Dummy_1991 + Dummy_1992 + Dummy_1993 + Dummy_1994 + Dummy_1995 + Dummy_1996 + Dummy_1997 + Dummy_1998 + Dummy_1999 + Dummy_2000 + Dummy_2001 + Dummy_2002 + Dummy_2003 + Dummy_2004 + Dummy_2005 + Dummy_2006 + Dummy_2007 + Dummy_2008 + Dummy_2009 + Dummy_2010 + Dummy_2011 + Dummy_2012 + Dummy_2013 + Dummy_2014 + Dummy_2015 + Dummy_2016 + Dummy_2017 + Dummy_2018 + Dummy_2019 + Dummy_2020 + Dummy_2021 + Dummy_2022 + Dummy_2023 + Dummy_giro_d_italia + Dummy_vuelta_a_espana + Dummy_dauphine + Dummy_tour_de_romandie + Dummy_volta_a_catalunya + Dummy_itzulia_basque_country + Dummy_tour_de_suisse + Dummy_tour_de_pologne + Dummy_paris_nice + Dummy_tirreno_adriatico + Dummy_stagetype_1 + Dummy_stagetype_2 + Dummy_stagetype_3 + Dummy_stagetype_4 + Dummy_stagetype_5', 
                      data=stage_c[(stage_c['Cluster'] == 1) | (stage_c['Cluster'] == 2)]).fit()
print(resultS12.summary())

# RHS: G1 if in G1/G2/G3
resultS123 = sm.ols(formula='Dummy_Cluster_1 ~ better_rider_around + Gap_12_larger1 + Gap_23_larger1 + Cluster_size_teams_winner + Cluster_size_teams_second + Cluster_size_teams_third + Dummy_1982 + Dummy_1983 + Dummy_1985 + Dummy_1986 + Dummy_1987 + Dummy_1988 + Dummy_1989 + Dummy_1990 + Dummy_1991 + Dummy_1992 + Dummy_1993 + Dummy_1994 + Dummy_1995 + Dummy_1996 + Dummy_1997 + Dummy_1998 + Dummy_1999 + Dummy_2000 + Dummy_2001 + Dummy_2002 + Dummy_2003 + Dummy_2004 + Dummy_2005 + Dummy_2006 + Dummy_2007 + Dummy_2008 + Dummy_2009 + Dummy_2010 + Dummy_2011 + Dummy_2012 + Dummy_2013 + Dummy_2014 + Dummy_2015 + Dummy_2016 + Dummy_2017 + Dummy_2018 + Dummy_2019 + Dummy_2020 + Dummy_2021 + Dummy_2022 + Dummy_2023 + Dummy_giro_d_italia + Dummy_vuelta_a_espana + Dummy_dauphine + Dummy_tour_de_romandie + Dummy_volta_a_catalunya + Dummy_itzulia_basque_country + Dummy_tour_de_suisse + Dummy_tour_de_pologne + Dummy_paris_nice + Dummy_tirreno_adriatico + Dummy_stagetype_1 + Dummy_stagetype_2 + Dummy_stagetype_3 + Dummy_stagetype_4 + Dummy_stagetype_5', 
                      data=stage_c[(stage_c['Cluster'] == 1) | (stage_c['Cluster'] == 2) | (stage_c['Cluster'] == 3)]).fit()
print(resultS123.summary())


                            OLS Regression Results                            
Dep. Variable:        Dummy_Cluster_1   R-squared:                       0.270
Model:                            OLS   Adj. R-squared:                  0.257
Method:                 Least Squares   F-statistic:                     20.35
Date:                Tue, 15 Oct 2024   Prob (F-statistic):          3.58e-189
Time:                        17:46:05   Log-Likelihood:                -1979.7
No. Observations:                3523   AIC:                             4087.
Df Residuals:                    3459   BIC:                             4482.
Df Model:                          63                                         
Covariance Type:            nonrobust                                         
                                   coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------
Intercept       

In [137]:
# Table 7: Linear Probability Model: Winning the Race from Group 1
stage_c_nsw= stage_c[stage_c['Cluster_size_teams_winner']!=1]

#LHS
resultS1x = sm.ols(formula='Win ~ better_rider_in_Cluster * Helper_in_Cluster + Teammates_behind + Gap_12_larger1 + Cluster_size_teams_winner + Cluster_size_teams_second + Dummy_1982 + Dummy_1983 + Dummy_1985 + Dummy_1986 + Dummy_1987 + Dummy_1988 + Dummy_1989 + Dummy_1990 + Dummy_1991 + Dummy_1992 + Dummy_1993 + Dummy_1994 + Dummy_1995 + Dummy_1996 + Dummy_1997 + Dummy_1998 + Dummy_1999 + Dummy_2000 + Dummy_2001 + Dummy_2002 + Dummy_2003 + Dummy_2004 + Dummy_2005 + Dummy_2006 + Dummy_2007 + Dummy_2008 + Dummy_2009 + Dummy_2010 + Dummy_2011 + Dummy_2012 + Dummy_2013 + Dummy_2014 + Dummy_2015 + Dummy_2016 + Dummy_2017 + Dummy_2018 + Dummy_2019 + Dummy_2020 + Dummy_2021 + Dummy_2022 + Dummy_2023 + Dummy_giro_d_italia + Dummy_vuelta_a_espana + Dummy_dauphine + Dummy_tour_de_romandie + Dummy_volta_a_catalunya + Dummy_itzulia_basque_country + Dummy_tour_de_suisse + Dummy_tour_de_pologne + Dummy_paris_nice + Dummy_tirreno_adriatico + Dummy_stagetype_1 + Dummy_stagetype_2 + Dummy_stagetype_3 + Dummy_stagetype_4 + Dummy_stagetype_5', 
                      data=stage_c_nsw[stage_c_nsw['Cluster'] == 1]).fit()
print(resultS1x.summary())

#Middle column
resultS1 = sm.ols(formula='Win ~ better_rider_in_Cluster + Helper_in_Cluster + Teammates_behind + Gap_12_larger1 + Cluster_size_teams_winner + Cluster_size_teams_second + Dummy_1982 + Dummy_1983 + Dummy_1985 + Dummy_1986 + Dummy_1987 + Dummy_1988 + Dummy_1989 + Dummy_1990 + Dummy_1991 + Dummy_1992 + Dummy_1993 + Dummy_1994 + Dummy_1995 + Dummy_1996 + Dummy_1997 + Dummy_1998 + Dummy_1999 + Dummy_2000 + Dummy_2001 + Dummy_2002 + Dummy_2003 + Dummy_2004 + Dummy_2005 + Dummy_2006 + Dummy_2007 + Dummy_2008 + Dummy_2009 + Dummy_2010 + Dummy_2011 + Dummy_2012 + Dummy_2013 + Dummy_2014 + Dummy_2015 + Dummy_2016 + Dummy_2017 + Dummy_2018 + Dummy_2019 + Dummy_2020 + Dummy_2021 + Dummy_2022 + Dummy_2023 + Dummy_giro_d_italia + Dummy_vuelta_a_espana + Dummy_dauphine + Dummy_tour_de_romandie + Dummy_volta_a_catalunya + Dummy_itzulia_basque_country + Dummy_tour_de_suisse + Dummy_tour_de_pologne + Dummy_paris_nice + Dummy_tirreno_adriatico + Dummy_stagetype_1 + Dummy_stagetype_2 + Dummy_stagetype_3 + Dummy_stagetype_4 + Dummy_stagetype_5', 
                   data=stage_c_nsw[stage_c_nsw['Cluster'] == 1]).fit()
print(resultS1.summary())

#RHS
#Hypothetical teams
# Note that there is randomness in how we define hypothetical teams. 
# Thus, the results presented in our paper cannot be replicated perfectly.

# Filter for captains only, no teammates in front or captain in cluster
stage_c_hyp = stage[(stage["Teammates_front_hyp"] == 0) & (stage["Captain_hyp_in_Cluster"] == 0)]
stage_c_hyp = stage_c_hyp[stage_c_hyp['Year'].astype(int) > 1980]  # Exclude data before 1980
stage_c_hyp.loc[:, 'Cluster_size_teams_hyp'] = stage_c_hyp.groupby(['Race_Stage', 'Cluster'])['Rider'].transform('nunique')

# Find winners and merge with stage data
winners = stage_c_hyp[stage_c_hyp['Place'] == 1][['Race_Stage', 'Cluster_size_teams']]
stage_c_hyp = pd.merge(stage_c_hyp, winners, on='Race_Stage', suffixes=('', '_winner'), how='left')

# Find second and third cluster data and merge with stage data
second = stage_c_hyp[stage_c_hyp['Cluster'] == 2][['Race_Stage', 'Cluster_size_teams']]
stage_c_hyp = pd.merge(stage_c_hyp, second, on='Race_Stage', suffixes=('', '_second'), how='left')

third = stage_c_hyp[stage_c_hyp['Cluster'] == 3][['Race_Stage', 'Cluster_size_teams']]
stage_c_hyp = pd.merge(stage_c_hyp, third, on='Race_Stage', suffixes=('', '_third'), how='left')

# Create dummies for gap size and standard deviation, drop duplicates
stage_c_hyp['Gap_12_larger1'] = (stage_c_hyp['Gap_Cluster12'] >= 60).astype(int)
stage_c_hyp['Gap_23_larger1'] = (stage_c_hyp['Gap_Cluster23'] >= 60).astype(int)
stage_c_hyp = stage_c_hyp.drop_duplicates()

# Repeat the process for hypothetical data: Find winners, second, and third cluster size
winners = stage_c_hyp[stage_c_hyp['Place'] == 1][['Race_Stage', 'Cluster_size_teams_hyp']]
stage_c_hyp = pd.merge(stage_c_hyp, winners, on='Race_Stage', suffixes=('', '_winner'), how='left')

second = stage_c_hyp[stage_c_hyp['Cluster'] == 2][['Race_Stage', 'Cluster_size_teams_hyp']]
stage_c_hyp = pd.merge(stage_c_hyp, second, on='Race_Stage', suffixes=('', '_second'), how='left')

third = stage_c_hyp[stage_c_hyp['Cluster'] == 3][['Race_Stage', 'Cluster_size_teams_hyp']]
stage_c_hyp = pd.merge(stage_c_hyp, third, on='Race_Stage', suffixes=('', '_third'), how='left')
stage_c_hyp = stage_c_hyp.drop_duplicates()

#No solo wins
stage_c_hyp_nsw= stage_c_hyp[stage_c_hyp['Cluster_size_winner']!=1]

# Winning the race for hypothetical teams
resultHyp = sm.ols(formula='Win ~ better_rider_in_Cluster + Helper_hyp_in_Cluster + Teammates_behind_hyp + Gap_12_larger1 + Cluster_size_teams_hyp_winner + Cluster_size_teams_hyp_second + Dummy_1982 + Dummy_1983 + Dummy_1985 + Dummy_1986 + Dummy_1987 + Dummy_1989 + Dummy_1990 + Dummy_1991 + Dummy_1992 + Dummy_1993 + Dummy_1994 + Dummy_1995 + Dummy_1996 + Dummy_1997 + Dummy_1998 + Dummy_1999 + Dummy_2000 + Dummy_2001 + Dummy_2002 + Dummy_2003 + Dummy_2004 + Dummy_2005 + Dummy_2006 + Dummy_2007 + Dummy_2008 + Dummy_2009 + Dummy_2010 + Dummy_2011 + Dummy_2012 + Dummy_2013 + Dummy_2014 + Dummy_2015 + Dummy_2016 + Dummy_2017 + Dummy_2018 + Dummy_2019 + Dummy_2020 + Dummy_2021 + Dummy_2022 + Dummy_2023 + Dummy_giro_d_italia + Dummy_vuelta_a_espana + Dummy_dauphine + Dummy_tour_de_romandie + Dummy_volta_a_catalunya + Dummy_itzulia_basque_country + Dummy_tour_de_suisse + Dummy_tour_de_pologne + Dummy_paris_nice + Dummy_tirreno_adriatico + Dummy_stagetype_1 + Dummy_stagetype_2 + Dummy_stagetype_3 + Dummy_stagetype_4 + Dummy_stagetype_5', 
                      data=stage_c_hyp_nsw[stage_c_hyp_nsw['Cluster'] == 1]).fit()
print(resultHyp.summary())


                            OLS Regression Results                            
Dep. Variable:                    Win   R-squared:                       0.097
Model:                            OLS   Adj. R-squared:                  0.048
Method:                 Least Squares   F-statistic:                     1.967
Date:                Tue, 15 Oct 2024   Prob (F-statistic):           1.67e-05
Time:                        17:46:05   Log-Likelihood:                -722.41
No. Observations:                1212   AIC:                             1573.
Df Residuals:                    1148   BIC:                             1899.
Df Model:                          63                                         
Covariance Type:            nonrobust                                         
                                                coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------

# Appendix

## Summary Statistics

In [138]:
# Table C.9: Summary Statistics Non-Dummies

# Create DataFrame 'races' for the calculations
races = pd.DataFrame()

# Calculate the mean for each race and each metric
races['Gap_Cluster12'] = stage.groupby(['Race_Stage'])['Gap_Cluster12'].mean()
races['Gap_Cluster23'] = stage.groupby(['Race_Stage'])['Gap_Cluster23'].mean()
races['Cluster_size_winner'] = stage.groupby(['Race_Stage'])['Cluster_size_winner'].mean()
races['Cluster_size_second'] = stage.groupby(['Race_Stage'])['Cluster_size_second'].mean()
races['Cluster_size_third'] = stage.groupby(['Race_Stage'])['Cluster_size_third'].mean()

# Combine the statistics into one DataFrame
summary_stats = pd.DataFrame({
    'Group 1 size': races['Cluster_size_winner'],
    'Group 2 size': races['Cluster_size_second'],
    'Group 3 size': races['Cluster_size_third'],
    'Gap between Groups 1 and 2': races['Gap_Cluster12'],
    'Gap between Groups 2 and 3': races['Gap_Cluster23']
})

# Use the .describe() function and filter for the relevant stats (mean, std, min, 50%, max)
summary_stats = summary_stats.describe().loc[['mean', 'std', 'min', '50%', 'max']]

# Rename index values to match your desired output
summary_stats.index = ['mean', 'std', 'min', '50%', 'max']

# Convert to LaTeX format and print
print(summary_stats.to_latex(index=True, float_format="%.2f"))

\begin{tabular}{lrrrrr}
\toprule
 & Group 1 size & Group 2 size & Group 3 size & Gap between Groups 1 and 2 & Gap between Groups 2 and 3 \\
\midrule
mean & 2.26 & 3.16 & 2.36 & 40.66 & 33.35 \\
std & 1.91 & 2.27 & 1.97 & 60.58 & 69.64 \\
min & 1.00 & 1.00 & 1.00 & 5.00 & 5.00 \\
50% & 2.00 & 2.00 & 1.00 & 22.00 & 14.00 \\
max & 12.00 & 12.00 & 11.00 & 853.00 & 1155.00 \\
\bottomrule
\end{tabular}



In [139]:
# Table C.10: Mean occurrence of Dummies

cl_one = stage[stage['Cluster'] == 1]
cl_two = stage[stage['Cluster'] == 2]
cl_three = stage[stage['Cluster'] == 3]

# Calculate mean values for each group
group1_mean = cl_one[['Star', 'better_rider_in_Cluster', 'Helper_in_Cluster', 'Teammates_behind']].mean()
group2_mean = cl_two[['Star', 'better_rider_in_Cluster', 'Helper_in_Cluster', 'Teammates_behind']].mean()
group3_mean = cl_three[['Star', 'better_rider_in_Cluster', 'Helper_in_Cluster']].mean()

# Calculate overall mean (across all groups)
overall_mean = stage[['Star', 'better_rider_in_Cluster', 'Helper_in_Cluster', 'Teammates_behind']].mean()

# Create DataFrame for the table
mean_occurrence = pd.DataFrame({
    'Group 1': group1_mean,
    'Group 2': group2_mean,
    'Group 3': group3_mean,
    'Overall': overall_mean
}).T

# Replace NaN values with 0.0 instead of an empty string (for compatibility)
mean_occurrence.fillna(0.0, inplace=True)

# Convert to LaTeX format
print(mean_occurrence[['Star', 'better_rider_in_Cluster', 'Helper_in_Cluster', 'Teammates_behind']].to_latex(index=True, float_format="%.3f"))



\begin{tabular}{lrrrr}
\toprule
 & Star & better_rider_in_Cluster & Helper_in_Cluster & Teammates_behind \\
\midrule
Group 1 & 0.284 & 0.262 & 0.051 & 0.122 \\
Group 2 & 0.269 & 0.351 & 0.066 & 0.093 \\
Group 3 & 0.193 & 0.271 & 0.053 & 0.000 \\
Overall & 0.251 & 0.301 & 0.058 & 0.108 \\
\bottomrule
\end{tabular}



## In-race data

In [140]:
# Load and clean data
km = pd.read_excel('1km_stats.xlsx')
km = km.drop(km.columns[0], axis=1).drop_duplicates()

# Calculate and print race summary
same = (km['same'] == 'y').sum()
total_races = len(km['same'])
share_same = same / total_races

# Convert rider and team columns to lists
for col in ['km_Riders_in_Cluster1', 'km_Riders_in_Cluster2', 'km_Riders_in_Cluster3', 'km_Teams_in_Cluster1', 'km_Teams_in_Cluster2', 'km_Teams_in_Cluster3']:
    km[col] = km[col].str.split(", ")

# Melt DataFrame for Riders and Teams
riders_df = km.melt(id_vars=['Race_Stage'], value_vars=['km_Riders_in_Cluster1', 'km_Riders_in_Cluster2', 'km_Riders_in_Cluster3'], var_name='km_Cluster', value_name='Rider')
teams_df = km.melt(id_vars=['Race_Stage'], value_vars=['km_Teams_in_Cluster1', 'km_Teams_in_Cluster2', 'km_Teams_in_Cluster3'], var_name='km_Cluster', value_name='Team')

# Function to extract the cluster number
def extract_last_number(s):
    numbers = [int(num) for num in re.findall(r'\d+', s)]
    return numbers[-1] if numbers else None

# Apply cluster extraction
riders_df['km_Cluster'] = riders_df['km_Cluster'].apply(extract_last_number).astype(int)
teams_df['km_Cluster'] = teams_df['km_Cluster'].apply(extract_last_number).astype(int)

# Merge Riders and Teams
merged_df = pd.merge(riders_df, teams_df, on=['Race_Stage', 'km_Cluster'])
df_1km = merged_df.explode(['Rider', 'Team'])

# Merge additional columns and sort
df_1km = pd.merge(df_1km, km[['Race_Stage', 'same', 'km_Cluster_size_winner', 'km_Cluster_size_second', 'km_Cluster_size_third']], on='Race_Stage', how='left')
df_1km = df_1km.sort_values(by=['Race_Stage', 'km_Cluster']).reset_index(drop=True)

# Merge with stage data
df = pd.merge(df_1km, stage[['Race_Stage', 'Rider', 'Cluster', 'Win', 'Place', 'Teammates_behind', 'Teammates_front', 
                              'Helper_in_Cluster', 'Captain_in_Cluster', 'Star', 'Cluster_size_teams_winner', 
                              'Cluster_size_teams_second', 'Cluster_size_teams_third', 'Dummy_Cluster_1', 
                              'Gap_12_larger1', 'Gap_23_larger1', 'better_rider_around', 
                              'better_rider_in_Cluster', 'Dummy_2020', 'Dummy_2021', 'Dummy_2022', 'Dummy_2023']], 
                  on=['Race_Stage', 'Rider'], how='left')

# Initialize new columns
df['km_Helper_in_Cluster'] = 0
df['km_Captain_in_Cluster'] = 0
df['km_Teammates_front'] = 0
df['km_Teammates_behind'] = 0

# Update teammate/helper information based on clusters
for group_name in df['Race_Stage'].unique():
    group_data = df[df['Race_Stage'] == group_name]
    for i in range(len(group_data)):
        for j in range(i + 1, len(group_data)):
            if (df.loc[group_data.index[i], 'km_Cluster'] == df.loc[group_data.index[j], 'km_Cluster']) and \
               (df.loc[group_data.index[i], 'Team'] == df.loc[group_data.index[j], 'Team']):
                df.loc[group_data.index[i], 'km_Helper_in_Cluster'] = 1
                df.loc[group_data.index[j], 'km_Captain_in_Cluster'] = 1
            if (df.loc[group_data.index[i], 'km_Cluster'] + 1 == df.loc[group_data.index[j], 'km_Cluster']) and \
               ((df.loc[group_data.index[i], 'Team'] == df.loc[group_data.index[j], 'Team']) or (df.loc[group_data.index[j], 'Team'] == 'peloton')):
                df.loc[group_data.index[i], 'km_Teammates_behind'] = 1
                df.loc[group_data.index[j], 'km_Teammates_front'] = 1

# Count various scenarios for winners and non-winners
winner_tb = len(df[(df['Place'] == 1) & (df['Teammates_behind'] == 1)])
nonwinner_tb = len(df[(df['Cluster'] == 1) & (df['Place'] != 1) & (df['Teammates_behind'] == 1)])
winner_tb_1km = len(df[(df['Place'] == 1) & (df['km_Teammates_behind'] == 1)])
nonwinner_tb_1km = len(df[(df['Cluster'] == 1) & (df['Place'] != 1) & (df['km_Teammates_behind'] == 1)])
helper_turns_tb = len(df[(df['Cluster'] == 1) & (df['km_Cluster'] == 1) & (df['Teammates_behind'] == 1) & (df['km_Helper_in_Cluster'] == 1)])
tb_turns_helper = len(df[(df['Cluster'] == 1) & (df['km_Cluster'] == 1) & (df['km_Teammates_behind'] == 1) & (df['Helper_in_Cluster'] == 1)])
winner_gets_tb = len(df[(df['Place'] == 1) & (df['Teammates_behind'] == 1) & (df['km_Teammates_behind'] != 1)])
nonwinner_gets_tb = len(df[(df['Place'] != 1) & (df['Cluster'] == 1) & (df['Teammates_behind'] == 1) & (df['km_Teammates_behind'] != 1)])

# Print summary statistics
print(f"The share of races where there is no change between the finish and 1km before is: {share_same:.2%} ({same} out of {total_races}).")
print(f"In our original dataset, the winner had a teammate behind in {winner_tb} cases, while non-winners in Cluster 1 had a teammate behind in {nonwinner_tb} cases.")
print(f"At 1km before the finish, the winner had a teammate behind in {winner_tb_1km} cases, and non-winners in Cluster 1 had a teammate behind in {nonwinner_tb_1km} cases.")
print(f"There are {helper_turns_tb} cases where a Cluster 1 rider had a teammate behind, who was a helper 1km before. In {tb_turns_helper} cases, the reverse occurred.")
print(f"Additionally, {winner_gets_tb} winners had a teammate behind at the finish but not 1km before, while {nonwinner_gets_tb} non-winners in Cluster 1 experienced the same.")

# Filter non-solo wins and run regressions
df_c = df[(df['Teammates_front'] == 0) & (df['Captain_in_Cluster'] == 0)].drop_duplicates()
df_c_nsw = df_c[df_c['Cluster_size_teams_winner'] != 1]

# Regression analysis

# Table C.11: Linear Probability Model: Finishing in Group 1 - In-Race Data
resultS12 = sm.ols('Dummy_Cluster_1 ~ better_rider_around + km_Teammates_behind + Gap_12_larger1 + Gap_23_larger1 + Cluster_size_teams_winner + Cluster_size_teams_second + Cluster_size_teams_third + Dummy_2021 + Dummy_2022 + Dummy_2023', data=df_c[(df_c['Cluster'] == 1) | (df_c['Cluster'] == 2)]).fit()
print(resultS12.summary())

# Table C.12: Linear Probability Model: Winning the Race from Group 1 - In-Race Data
resultS1 = sm.ols('Win ~ better_rider_in_Cluster + km_Helper_in_Cluster + km_Teammates_behind + Gap_12_larger1 + Cluster_size_teams_winner + Cluster_size_teams_second + Dummy_2021 + Dummy_2022 + Dummy_2023', data=df_c_nsw[df_c_nsw['Cluster'] == 1]).fit()
print(resultS1.summary())


The share of races where there is no change between the finish and 1km before is: 53.47% (54 out of 101).
In our original dataset, the winner had a teammate behind in 16 cases, while non-winners in Cluster 1 had a teammate behind in 9 cases.
At 1km before the finish, the winner had a teammate behind in 17 cases, and non-winners in Cluster 1 had a teammate behind in 10 cases.
There are 5 cases where a Cluster 1 rider had a teammate behind, who was a helper 1km before. In 1 cases, the reverse occurred.
Additionally, 2 winners had a teammate behind at the finish but not 1km before, while 3 non-winners in Cluster 1 experienced the same.
                            OLS Regression Results                            
Dep. Variable:        Dummy_Cluster_1   R-squared:                       0.293
Model:                            OLS   Adj. R-squared:                  0.278
Method:                 Least Squares   F-statistic:                     19.43
Date:                Tue, 15 Oct 2024   Pro

## Additional analyses

### Fewer Stars

In [141]:
# Table C.13: Linear Probability Model: Finishing in Group 1 - Fewer Stars
# Table C.14: Linear Probability Model: Winning the Race from Group 1 - Fewer Stars

#Change Section "Stars"
# Here, do the same analysis as for Tables 6 and 7 after adjusting the definition of STARS in the following way:

# Loop through years to create dummy variables for each year and identify stars
for i in range(1981, 2024):
    stage[f'Dummy_{i}'] = 0
    stage.loc[stage['Year'] == i, f'Dummy_{i}'] = 1
    threshold_up = stage.loc[stage['Year'] == i, 'Score'].quantile(0.90)
    
    stage.loc[stage['Year'] == i, 'Star'] = (stage.loc[stage['Year'] == i, 'Score'] >= threshold_up).astype(int)
    stage.loc[stage['Year'] == i, 'not_a_Star'] = (stage.loc[stage['Year'] == i, 'Score'] < threshold_up).astype(int)

# other than that, use the same code as above

### Smaller Groups

In [142]:
# Table C.15: Linear Probability Model: Finishing in Group 1 - Smaller Groups
# Table C.16: Linear Probability Model: Winning the Race from Group 1 - Smaller Groups

#Change Section "Groups"
# Here, do the same analysis as for Tables 6 and 7 after adjusting the definition of GROUPS in the following way:

# Iterate through unique race stages to define clusters and teammate roles
for group_name in stage['Race_Stage'].unique():
    group_data = stage[stage['Race_Stage'] == group_name]
    
    # Define clusters based on time gaps
    for i in range(1, len(group_data)):
        gap_difference = group_data.iloc[i]['Gap'] - group_data.iloc[i - 1]['Gap']
        if gap_difference > 0:  # 1+ seconds gap creates a new cluster
            stage.loc[group_data.index[i], 'Cluster'] = stage.loc[group_data.index[i - 1], 'Cluster'] + 1
            stage.loc[group_data.index[i], 'Gap_front'] = gap_difference
        else:
            stage.loc[group_data.index[i], 'Gap_front'] = 0
            stage.loc[group_data.index[i], 'Cluster'] = stage.loc[group_data.index[i - 1], 'Cluster']
    
    # Update cluster sizes and teammate information
    for i in range(len(group_data)):
        for j in range(i + 1, len(group_data)):
            same_cluster = stage.loc[group_data.index[i], 'Cluster'] == stage.loc[group_data.index[j], 'Cluster']
            same_team = stage.loc[group_data.index[i], 'Team'] == stage.loc[group_data.index[j], 'Team']
            if same_cluster:
                stage.loc[group_data.index[i], 'Cluster_size'] += 1
                stage.loc[group_data.index[j], 'Cluster_size'] += 1
                if same_team:
                    # Mark teammates in same cluster
                    stage.loc[group_data.index[i], 'Helper_in_Cluster'] = 1
                    stage.loc[group_data.index[j], 'Captain_in_Cluster'] = 1
                    stage.loc[group_data.index[i], 'Teammates'] = 1
                    stage.loc[group_data.index[j], 'Teammates'] = 1
                else:
                    # Non-teammates in the same cluster
                    pass
                
            # Mark teammates in neighboring clusters
            if same_team and stage.loc[group_data.index[i], 'Cluster'] + 1 == stage.loc[group_data.index[j], 'Cluster']:
                stage.loc[group_data.index[i], 'Teammates_behind'] = 1
                stage.loc[group_data.index[j], 'Teammates_front'] = 1
            
            if same_team and stage.loc[group_data.index[i], 'Cluster'] + 2 == stage.loc[group_data.index[j], 'Cluster']:
                stage.loc[group_data.index[j], 'Teammates_front'] = 1
            
            # Hypothetical teammates in same cluster
            if same_cluster and stage.loc[group_data.index[i], 'hyp_team'] == stage.loc[group_data.index[j], 'hyp_team']:
                stage.loc[group_data.index[i], 'Helper_hyp_in_Cluster'] = 1
                stage.loc[group_data.index[j], 'Captain_hyp_in_Cluster'] = 1
                
            # Hypothetical teammates in neighboring clusters
            if stage.loc[group_data.index[i], 'Cluster'] + 1 == stage.loc[group_data.index[j], 'Cluster'] and stage.loc[group_data.index[i], 'hyp_team'] == stage.loc[group_data.index[j], 'hyp_team']:
                stage.loc[group_data.index[i], 'Teammates_behind_hyp'] = 1
                stage.loc[group_data.index[j], 'Teammates_front_hyp'] = 1

# other than that, use the same code as above

### Hypothetical teammates

In [143]:
# Table C.17: Linear Probability Model: Winning the Race from Group 1

# Only the RHS is new (for LHS, see Table 7)
resultHypx = sm.ols(formula='Win ~ better_rider_in_Cluster * Helper_hyp_in_Cluster + Teammates_behind_hyp + Gap_12_larger1 + Cluster_size_teams_hyp_winner + Cluster_size_teams_hyp_second + Dummy_1982 + Dummy_1983 + Dummy_1985 + Dummy_1986 + Dummy_1987  + Dummy_1989 + Dummy_1990 + Dummy_1991 + Dummy_1992 + Dummy_1993 + Dummy_1994 + Dummy_1995 + Dummy_1996 + Dummy_1997 + Dummy_1998 + Dummy_1999 + Dummy_2000 + Dummy_2001 + Dummy_2002 + Dummy_2003 + Dummy_2004 + Dummy_2005 + Dummy_2006 + Dummy_2007 + Dummy_2008 + Dummy_2009 + Dummy_2010 + Dummy_2011 + Dummy_2012 + Dummy_2013 + Dummy_2014 + Dummy_2015 + Dummy_2016 + Dummy_2017 + Dummy_2018 + Dummy_2019 + Dummy_2020 + Dummy_2021 + Dummy_2022 + Dummy_2023 + Dummy_giro_d_italia + Dummy_vuelta_a_espana + Dummy_dauphine + Dummy_tour_de_romandie + Dummy_volta_a_catalunya + Dummy_itzulia_basque_country + Dummy_tour_de_suisse + Dummy_tour_de_pologne + Dummy_paris_nice + Dummy_tirreno_adriatico + Dummy_stagetype_1 + Dummy_stagetype_2 + Dummy_stagetype_3 + Dummy_stagetype_4 + Dummy_stagetype_5', 
                      data=stage_c_hyp_nsw[stage_c_hyp_nsw['Cluster'] == 1]).fit()
print(resultHypx.summary())


                            OLS Regression Results                            
Dep. Variable:                    Win   R-squared:                       0.103
Model:                            OLS   Adj. R-squared:                  0.048
Method:                 Least Squares   F-statistic:                     1.872
Date:                Tue, 15 Oct 2024   Prob (F-statistic):           8.07e-05
Time:                        17:46:08   Log-Likelihood:                -652.86
No. Observations:                1071   AIC:                             1432.
Df Residuals:                    1008   BIC:                             1745.
Df Model:                          62                                         
Covariance Type:            nonrobust                                         
                                                    coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------

## Logistic regressions

In [144]:
# Table C.18: Logistic Probability Model: Finishing in Group 1

# LHS: G1 if in G1/G2
resultS12L = sm.logit(formula = 'Dummy_Cluster_1 ~ better_rider_around + Teammates_behind  + Gap_12_larger1 + Gap_23_larger1 + Cluster_size_teams_winner + Cluster_size_teams_second +Cluster_size_teams_third+ Dummy_1982 + Dummy_1983 + Dummy_1985 + Dummy_1986 + Dummy_1987 + Dummy_1988 + Dummy_1989 + Dummy_1990 + Dummy_1991 + Dummy_1992 + Dummy_1993 + Dummy_1994 + Dummy_1995 + Dummy_1996 + Dummy_1997 + Dummy_1998 + Dummy_1999 + Dummy_2000 + Dummy_2001+Dummy_2002+Dummy_2003+Dummy_2004+Dummy_2005+Dummy_2006+Dummy_2007+Dummy_2008+Dummy_2009+Dummy_2010+Dummy_2011+Dummy_2012+Dummy_2013+Dummy_2014+Dummy_2015+Dummy_2016+Dummy_2017+Dummy_2018+Dummy_2019+Dummy_2020+Dummy_2021+Dummy_2022+Dummy_2023 + Dummy_giro_d_italia + Dummy_vuelta_a_espana +Dummy_dauphine + Dummy_tour_de_romandie + Dummy_volta_a_catalunya + Dummy_itzulia_basque_country + Dummy_tour_de_suisse + Dummy_tour_de_pologne + Dummy_paris_nice +Dummy_tirreno_adriatico +Dummy_stagetype_1+Dummy_stagetype_2+Dummy_stagetype_2+Dummy_stagetype_3+Dummy_stagetype_4+Dummy_stagetype_5', data=stage_c[(stage_c['Cluster']==1) |(stage_c['Cluster']==2)]).fit()#cov_type='cluster', cov_kwds={'groups': stage_c[(stage_c['Cluster']==1) |(stage_c['Cluster']==2)]['Race']})
print(resultS12L.summary())

# RHS: G1 if in G1/G2/G3
resultS123L = sm.logit(formula = 'Dummy_Cluster_1 ~ better_rider_around+ Gap_12_larger1+ Gap_23_larger1 + Cluster_size_teams_winner + Cluster_size_teams_second +Cluster_size_teams_third + Dummy_1982 + Dummy_1983 + Dummy_1985 + Dummy_1986 + Dummy_1987 + Dummy_1988 + Dummy_1989 + Dummy_1990 + Dummy_1991 + Dummy_1992 + Dummy_1993 + Dummy_1994 + Dummy_1995 + Dummy_1996 + Dummy_1997 + Dummy_1998 + Dummy_1999 + Dummy_2000 + Dummy_2001+Dummy_2002+Dummy_2003+Dummy_2004+Dummy_2005+Dummy_2006+Dummy_2007+Dummy_2008+Dummy_2009+Dummy_2010+Dummy_2011+Dummy_2012+Dummy_2013+Dummy_2014+Dummy_2015+Dummy_2016+Dummy_2017+Dummy_2018+Dummy_2019+Dummy_2020+Dummy_2021+Dummy_2022+Dummy_2023 + Dummy_giro_d_italia + Dummy_vuelta_a_espana +Dummy_dauphine + Dummy_tour_de_romandie + Dummy_volta_a_catalunya + Dummy_itzulia_basque_country + Dummy_tour_de_suisse + Dummy_tour_de_pologne + Dummy_paris_nice +Dummy_tirreno_adriatico +Dummy_stagetype_1+Dummy_stagetype_2+Dummy_stagetype_2+Dummy_stagetype_3+Dummy_stagetype_4+Dummy_stagetype_5', data=stage_c[(stage_c['Cluster']==1) |(stage_c['Cluster']==2) |(stage_c['Cluster']==3)]).fit()#cov_type='cluster', cov_kwds={'groups': stage_c[(stage_c['Cluster']==1) |(stage_c['Cluster']==2) |(stage_c['Cluster']==3)]['Race']})
print(resultS123L.summary())

Optimization terminated successfully.
         Current function value: 0.526260
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:        Dummy_Cluster_1   No. Observations:                 3523
Model:                          Logit   Df Residuals:                     3459
Method:                           MLE   Df Model:                           63
Date:                Tue, 15 Oct 2024   Pseudo R-squ.:                  0.2339
Time:                        17:46:09   Log-Likelihood:                -1854.0
converged:                       True   LL-Null:                       -2420.2
Covariance Type:            nonrobust   LLR p-value:                8.404e-196
                                   coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------
Intercept                       -0.2329      0.394     -0.591      0.554

In [145]:
# Table C.19: Logistic Probability Model: Winning the Race from Group 1

#LHS
resultS1Lx = sm.logit(formula = 'Win ~ better_rider_in_Cluster *Helper_in_Cluster + Teammates_behind + Gap_12_larger1 + Cluster_size_teams_winner + Cluster_size_teams_second+Dummy_1982 + Dummy_1983 + Dummy_1985 + Dummy_1986 + Dummy_1987 + Dummy_1988 + Dummy_1989 + Dummy_1990 + Dummy_1991 + Dummy_1992 + Dummy_1993 + Dummy_1994 + Dummy_1995 + Dummy_1996 + Dummy_1997 + Dummy_1998 + Dummy_1999 + Dummy_2000 + Dummy_2001+Dummy_2002+Dummy_2003+Dummy_2004+Dummy_2005+Dummy_2006+Dummy_2007+Dummy_2008+Dummy_2009+Dummy_2010+Dummy_2011+Dummy_2012+Dummy_2013+Dummy_2014+Dummy_2015+Dummy_2016+Dummy_2017+Dummy_2018+Dummy_2019+Dummy_2020+Dummy_2021+Dummy_2022+Dummy_2023 + Dummy_giro_d_italia + Dummy_vuelta_a_espana +Dummy_dauphine + Dummy_tour_de_romandie + Dummy_volta_a_catalunya + Dummy_itzulia_basque_country + Dummy_tour_de_suisse + Dummy_tour_de_pologne + Dummy_paris_nice +Dummy_tirreno_adriatico +Dummy_stagetype_1+Dummy_stagetype_2+Dummy_stagetype_2+Dummy_stagetype_3+Dummy_stagetype_4+Dummy_stagetype_5', data=stage_c_nsw[stage_c_nsw['Cluster']==1]).fit()#cov_type='cluster', cov_kwds={'groups': stage_c[stage_c['Cluster']==1]['Race']})
print(resultS1Lx.summary())

#Middle column
resultS1L = sm.logit(formula = 'Win ~ better_rider_in_Cluster +Helper_in_Cluster + Teammates_behind + Gap_12_larger1 + Cluster_size_teams_winner + Cluster_size_teams_second+Dummy_1982 + Dummy_1983 + Dummy_1985 + Dummy_1986 + Dummy_1987 + Dummy_1988 + Dummy_1989 + Dummy_1990 + Dummy_1991 + Dummy_1992 + Dummy_1993 + Dummy_1994 + Dummy_1995 + Dummy_1996 + Dummy_1997 + Dummy_1998 + Dummy_1999 + Dummy_2000 + Dummy_2001+Dummy_2002+Dummy_2003+Dummy_2004+Dummy_2005+Dummy_2006+Dummy_2007+Dummy_2008+Dummy_2009+Dummy_2010+Dummy_2011+Dummy_2012+Dummy_2013+Dummy_2014+Dummy_2015+Dummy_2016+Dummy_2017+Dummy_2018+Dummy_2019+Dummy_2020+Dummy_2021+Dummy_2022+Dummy_2023 + Dummy_giro_d_italia + Dummy_vuelta_a_espana +Dummy_dauphine + Dummy_tour_de_romandie + Dummy_volta_a_catalunya + Dummy_itzulia_basque_country + Dummy_tour_de_suisse + Dummy_tour_de_pologne + Dummy_paris_nice +Dummy_tirreno_adriatico +Dummy_stagetype_1+Dummy_stagetype_2+Dummy_stagetype_2+Dummy_stagetype_3+Dummy_stagetype_4+Dummy_stagetype_5', data=stage_c_nsw[stage_c_nsw['Cluster']==1]).fit()#cov_type='cluster', cov_kwds={'groups': stage_c[stage_c['Cluster']==1]['Race']})
print(resultS1L.summary())

#RHS
# Winning the race for hypothetical teams
resultHypL = sm.logit(formula = 'Win ~ better_rider_in_Cluster+Helper_hyp_in_Cluster + Teammates_behind_hyp + Gap_12_larger1 + Cluster_size_teams_hyp_winner + Cluster_size_teams_hyp_second+Dummy_1982 + Dummy_1983 + Dummy_1985 + Dummy_1986 + Dummy_1987 + Dummy_1988 + Dummy_1989 + Dummy_1990 + Dummy_1991 + Dummy_1992 + Dummy_1993 + Dummy_1994 + Dummy_1995 + Dummy_1996 + Dummy_1997 + Dummy_1998 + Dummy_1999 + Dummy_2000 + Dummy_2001+Dummy_2002+Dummy_2003+Dummy_2004+Dummy_2005+Dummy_2006+Dummy_2007+Dummy_2008+Dummy_2009+Dummy_2010+Dummy_2011+Dummy_2012+Dummy_2013+Dummy_2014+Dummy_2015+Dummy_2016+Dummy_2017+Dummy_2018+Dummy_2019+Dummy_2020+Dummy_2021+Dummy_2022+Dummy_2023 + Dummy_giro_d_italia + Dummy_vuelta_a_espana +Dummy_dauphine + Dummy_tour_de_romandie + Dummy_volta_a_catalunya + Dummy_itzulia_basque_country + Dummy_tour_de_suisse + Dummy_tour_de_pologne + Dummy_paris_nice +Dummy_tirreno_adriatico +Dummy_stagetype_1+Dummy_stagetype_2+Dummy_stagetype_2+Dummy_stagetype_3+Dummy_stagetype_4+Dummy_stagetype_5', data=stage_c_hyp_nsw[stage_c_hyp_nsw['Cluster']==1]).fit()#cov_type='cluster', cov_kwds={'groups': stage_c[stage_c['Cluster']==1]['Race']})
print(resultHypL.summary())


Optimization terminated successfully.
         Current function value: 0.563631
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:                    Win   No. Observations:                 1212
Model:                          Logit   Df Residuals:                     1148
Method:                           MLE   Df Model:                           63
Date:                Tue, 15 Oct 2024   Pseudo R-squ.:                 0.08890
Time:                        17:46:10   Log-Likelihood:                -683.12
converged:                       True   LL-Null:                       -749.77
Covariance Type:            nonrobust   LLR p-value:                 5.868e-07
                                                coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------
Intercept                                     

## One-day races

### Create dataframe - One-day races

In [146]:
#ONEDAY RACES
oneday = pd.read_excel('one_day_races_nf15.xlsx')
oneday = oneday.drop(oneday.columns[0], axis=1)
oneday = oneday.drop(columns=['Team_Score'])
oneday = oneday.drop_duplicates()

oneday['Helper_in_Cluster'] = 0
oneday['Helper_hyp_in_Cluster'] = 0
oneday['Captain_hyp_in_Cluster'] = 0
oneday['Captain_in_Cluster'] = 0
oneday['Teammates_behind'] = 0
oneday['Teammates_behind_hyp'] = 0
oneday['Teammates_front'] = 0
oneday['Teammates_front_hyp'] = 0
oneday['Teammates'] = 0
oneday['Cluster'] = 1
oneday['Cluster_size'] = 1
oneday['Cluster_size_teams'] = 1
oneday['Cluster_size_teams_hyp'] = 1
oneday['Win'] = 0
oneday['Star'] = 0
oneday['Star_other_in_Cluster'] = 0
oneday['not_a_Star'] = 0
oneday['Star_other'] = 0 
oneday['Star_other_team'] = 0
oneday['Star_my_team'] = 0
oneday['Star_of_Cluster'] = 0
oneday['Star_other_team_in_Cluster'] = 0
oneday['Star_my_team_in_Cluster'] = 0
oneday['eliminate'] = 0
oneday.loc[oneday['Race'] == 'Flèche Wallone', 'Race']='FW'
oneday.loc[oneday['Race'] == 'San Sebastian', 'Race']='SS'
oneday['Race_Year'] = oneday['Race'] + oneday['Year'].astype(str)
oneday['hyp_team'] = np.random.randint(1, 23, size=len(oneday))

print('We downloaded a total of', len(oneday['Race_Year'].unique()), 'one-day races.')

oneday.loc[oneday['Place'] == 1, 'Win'] = 1
oneday['Year'] = oneday['Year'].astype(int)
for i in range(1981, 2024):
    oneday.loc[oneday['Year'] == i, 'Dummy_'+str(i)] = 1
    oneday.loc[oneday['Year'] != i, 'Dummy_'+str(i)] = 0
    threshold_up = oneday.loc[oneday['Year'] == i, "Score"].quantile(80/100)
    threshold_down = oneday.loc[oneday['Year'] == i, "Score"].quantile(20/100)
    oneday.loc[oneday['Year'] == i, 'Star'] = (oneday.loc[oneday['Year'] == i, "Score"] >= threshold_up).astype(int)
for s in oneday['Race'].unique():
    oneday.loc[oneday['Race'] == s, 'Dummy_'+s] = 1
    oneday.loc[oneday['Race'] != s, 'Dummy_'+s] = 0

# Iterate through unique Races
group_names = oneday['Race_Year'].unique()
for group_name in group_names:
    group_data = oneday[oneday['Race_Year'] == group_name]
    for i in range(1, len(group_data)):
        gap_difference = group_data.iloc[i]['Gap'] - group_data.iloc[i - 1]['Gap']
        #If difference to rider in front is at least 5sec then next group
        if gap_difference > 4:
            oneday.loc[group_data.index[i], 'Cluster'] = oneday.loc[group_data.index[i - 1], 'Cluster'] + 1
            oneday.loc[group_data.index[i], 'Gap_front'] = gap_difference
        else:
            oneday.loc[group_data.index[i], 'Gap_front'] = 0
            oneday.loc[group_data.index[i], 'Cluster'] = oneday.loc[group_data.index[i - 1], 'Cluster']
    for i in range(0, len(group_data)):        
        for j in range(i+1, len(group_data)):
            if (oneday.loc[group_data.index[i], 'Cluster'] == oneday.loc[group_data.index[j], 'Cluster']) and (oneday.loc[group_data.index[i], 'Team'] != oneday.loc[group_data.index[j], 'Team']):
                oneday.loc[group_data.index[i], 'Cluster_size'] +=1
                oneday.loc[group_data.index[j], 'Cluster_size'] +=1
            if (oneday.loc[group_data.index[i], 'Cluster'] == oneday.loc[group_data.index[j], 'Cluster']) and (oneday.loc[group_data.index[i], 'Team'] == oneday.loc[group_data.index[j], 'Team']):
                oneday.loc[group_data.index[i], 'Cluster_size'] +=1
                oneday.loc[group_data.index[j], 'Cluster_size'] +=1
                oneday.loc[group_data.index[i], 'Helper_in_Cluster'] = 1
                oneday.loc[group_data.index[j], 'Captain_in_Cluster'] = 1
                oneday.loc[group_data.index[i], 'Teammates'] = 1
                oneday.loc[group_data.index[j], 'Teammates'] = 1
            if (oneday.loc[group_data.index[i], 'Cluster']+1 == oneday.loc[group_data.index[j], 'Cluster']) and (oneday.loc[group_data.index[i], 'Team'] == oneday.loc[group_data.index[j], 'Team']):
                oneday.loc[group_data.index[i], 'Teammates_behind'] = 1
                oneday.loc[group_data.index[j], 'Teammates_front'] = 1
                oneday.loc[group_data.index[i], 'Teammates'] = 1
                oneday.loc[group_data.index[j], 'Teammates'] = 1
            if (oneday.loc[group_data.index[i], 'Cluster']+2 == oneday.loc[group_data.index[j], 'Cluster']) and (oneday.loc[group_data.index[i], 'Team'] == oneday.loc[group_data.index[j], 'Team']):
                oneday.loc[group_data.index[j], 'Teammates_front'] = 1
            if (oneday.loc[group_data.index[i], 'Cluster'] == oneday.loc[group_data.index[j], 'Cluster']) and (oneday.loc[group_data.index[i], 'hyp_team'] == oneday.loc[group_data.index[j], 'hyp_team']):
                oneday.loc[group_data.index[i], 'Helper_hyp_in_Cluster'] = 1
                oneday.loc[group_data.index[j], 'Captain_hyp_in_Cluster'] = 1
            if (oneday.loc[group_data.index[i], 'Cluster']+1 == oneday.loc[group_data.index[j], 'Cluster']) and (oneday.loc[group_data.index[i], 'hyp_team'] == oneday.loc[group_data.index[j], 'hyp_team']):
                oneday.loc[group_data.index[i], 'Teammates_behind_hyp'] = 1
                oneday.loc[group_data.index[j], 'Teammates_front_hyp'] = 1
            if (oneday.loc[group_data.index[i], 'Cluster']+2 == oneday.loc[group_data.index[j], 'Cluster']) and (oneday.loc[group_data.index[i], 'hyp_team'] == oneday.loc[group_data.index[j], 'hyp_team']):
                oneday.loc[group_data.index[j], 'Teammates_front_hyp'] = 1

oneday.loc[:, 'Cluster_size_teams'] = oneday.groupby(['Race_Year', 'Cluster'])['Team'].transform('nunique')
oneday = oneday.copy()
oneday = oneday.drop_duplicates()

for s in oneday['Cluster'].unique():
    oneday.loc[oneday['Cluster'] == 1, 'Dummy_Cluster_1'] = 1
    oneday.loc[oneday['Cluster'] != 1, 'Dummy_Cluster_1'] = 0

# Group by 'Race_Year' and count the unique teams in each group
teams_per_year= oneday.groupby('Race_Year')['Team'].nunique()
# Calculate the mean number of distinct teams per Race_Year
mean_teams_per_year = teams_per_year.mean()
# Group by 'Race_Year' and count the unique hyp_teams in each group
teams_per_year = oneday.groupby('Race_Year')['hyp_team'].nunique()
# Calculate the mean number of distinct teams per Race_Year
hyp_mean_teams_per_year = teams_per_year.mean()

# Find the winners for each group and include 'Cluster_size_winner' and same for second
winners = oneday[oneday['Place'] == 1][['Race_Year', 'Cluster_size', 'Cluster_size_teams']]
oneday = pd.merge(oneday, winners, on='Race_Year', suffixes=('', '_winner'), how='left')
second = oneday[oneday['Cluster'] == 2][['Race_Year', 'Cluster_size', 'Cluster_size_teams']]
oneday = pd.merge(oneday, second, on='Race_Year', suffixes=('', '_second'), how='left')
third = oneday[oneday['Cluster'] == 3][['Race_Year', 'Cluster_size', 'Cluster_size_teams']]
oneday = pd.merge(oneday, third, on='Race_Year', suffixes=('', '_third'), how='left')

#we need 3 complete clusters and we need to drop all races where the first two groups consist of only one team each
for s in oneday['Race_Year'].unique():
    group_data = oneday.loc[oneday['Race_Year'] == s]
    for i in range(len(group_data)):
        if (oneday.loc[group_data.index[i], 'Place'] == 15) & ((oneday.loc[group_data.index[i], 'Cluster'] == 1) | (oneday.loc[group_data.index[i], 'Cluster'] == 2) | (oneday.loc[group_data.index[i], 'Cluster'] == 3)):
        #if (oneday.loc[group_data.index[i], 'Place'] == 15) & ((oneday.loc[group_data.index[i], 'Cluster'] == 1) | (oneday.loc[group_data.index[i], 'Cluster'] == 2)):
            oneday.loc[group_data.index[i], 'eliminate'] = 1
        if  (oneday.loc[group_data.index[i], 'Cluster_size_teams_winner'] == 1) & (oneday.loc[group_data.index[i], 'Cluster_size_teams_second'] == 1):
            oneday.loc[group_data.index[i], 'eliminate'] = 1
# Identify unique values of 'Race_Year' where 'eliminate' is already 1
eliminate_race_years = oneday.loc[oneday['eliminate'] == 1, 'Race_Year'].unique()

# Update 'eliminate' column for all rows with a unique value of 'Race_Year'
oneday.loc[oneday['Race_Year'].isin(eliminate_race_years), 'eliminate'] = 1
oneday = oneday[oneday['eliminate'] != 1]
oneday = oneday.copy()
# Filter out riders where Cluster <= 3
oneday = oneday.drop_duplicates()
stage_filtered = oneday[oneday['Cluster'] <= 3].copy()
# Reset the index of the filtered DataFrame
stage_filtered.reset_index(drop=True, inplace=True)
oneday=stage_filtered

oneday = oneday.drop_duplicates()
#other Star in Cluster   
oneday['Cluster_id'] = oneday['Race_Year']+oneday['Cluster'].astype(str)
grouped_data = oneday.groupby('Cluster_id')['Star']
sum_star = grouped_data.transform('sum')
oneday['Star_other_in_Cluster'] = (((sum_star >= 2) & (oneday['Star'] == 1)) |((sum_star >= 1) & (oneday['Star'] != 1))).astype(int)
#other Star in Cluster from another team   
for c in oneday['Cluster_id'].unique():
    group_data = oneday.loc[oneday['Cluster_id'] == c]
    for i in range(len(group_data)):
        for j in range(len(group_data)):
            if (oneday.loc[group_data.index[i], 'Team'] != oneday.loc[group_data.index[j], 'Team']) and (oneday.loc[group_data.index[j], 'Star'] == 1):
                oneday.loc[group_data.index[i], 'Star_other_team_in_Cluster'] = 1
            if (oneday.loc[group_data.index[i], 'Team'] == oneday.loc[group_data.index[j], 'Team']) and (oneday.loc[group_data.index[j], 'Star'] == 1) and (i!=j):
                oneday.loc[group_data.index[i], 'Star_my_team_in_Cluster'] = 1
oneday.drop('Cluster_id', axis=1, inplace=True)
oneday = oneday.drop_duplicates()

for s in oneday['Race_Year'].unique():
    group_data = oneday.loc[oneday['Race_Year'] == s]
    for i in range(len(group_data)):
        #let us define Star_other as a dummy that indicates whether one of the other riders in the first two clusters has 'Star score'
        oneday.loc[group_data.index[i], 'Star_other'] = (np.sum(group_data[(group_data['Cluster']==1) | (group_data['Cluster']==2)]['Star']) > oneday.loc[group_data.index[i]]["Star"]).astype(int)   
        oneday.loc[group_data.index[i], 'Star_in_Cluster1'] = (np.sum(group_data[group_data['Cluster']==1]['Star']) >0).astype(int)   
        oneday.loc[group_data.index[i], 'Star_in_Cluster2'] = (np.sum(group_data[group_data['Cluster']==2]['Star']) >0).astype(int)   
        oneday.loc[group_data.index[i], 'Winner_is_Star'] = (np.sum(group_data[group_data['Win']==1]['Star']) >0).astype(int)
        oneday.loc[group_data.index[i], 'Gap_Cluster12'] = (group_data[group_data['Cluster']==2]['Gap_front']).max()
        oneday.loc[group_data.index[i], 'Gap_Cluster23'] = (group_data[group_data['Cluster']==3]['Gap_front']).max()
        oneday.loc[group_data.index[i], 'Helper_in_Cluster2'] = (np.sum(group_data[group_data['Cluster']==2]['Helper_in_Cluster']) >0).astype(int)
        oneday.loc[group_data.index[i], 'Winner_is_Satellite'] = (np.sum(group_data[group_data['Win']==1]['Teammates_behind']) >0).astype(int)
        oneday.loc[group_data.index[i], 'Cluster2_std'] = group_data[group_data['Cluster']==2]['Score'].std()
        if (oneday.loc[group_data.index[i], 'Team'] != oneday.loc[group_data.index[j], 'Team']) and ((oneday.loc[group_data.index[j], 'Cluster'] == 1) | (oneday.loc[group_data.index[j], 'Cluster'] == 2)) and (oneday.loc[group_data.index[j], 'Star'] == 1) and (i!=j):
            oneday.loc[group_data.index[i], 'Star_other_team'] = 1
        if (oneday.loc[group_data.index[i], 'Team'] == oneday.loc[group_data.index[j], 'Team']) and ((oneday.loc[group_data.index[j], 'Cluster'] == 1) | (oneday.loc[group_data.index[j], 'Cluster'] == 2)) and (oneday.loc[group_data.index[j], 'Star'] == 1) and (i!=j):
            oneday.loc[group_data.index[i], 'Star_my_team'] = 1
oneday = oneday.drop_duplicates()

oneday['no_Star_my_team_in_Cluster'] = 1 - oneday['Star_my_team_in_Cluster']
oneday['no_Star_other_team_in_Cluster'] = 1 - oneday['Star_other_team_in_Cluster']
oneday['no_Star_other_team'] = 1 - oneday['Star_other_team']
oneday['no_Star'] = 1 - oneday['Star']
#we want a variable indicating whether there is a better rider in the group 
#(i.e., dummy equal to 1 if rider is not a Star but Star in group exists)
oneday['better_rider_in_Cluster'] = oneday.apply(lambda row: 1 if row['Star_other_team_in_Cluster'] == 1 and row['Star'] == 0 else 0, axis=1)
oneday['better_rider_around'] = oneday.apply(lambda row: 1 if row['Star_other_team'] == 1 and row['Star'] == 0 else 0, axis=1)

#find solo wins
oneday['Solo_Win'] = (oneday['Cluster_size_winner']==1).astype(int)
#Dummy for Gap Size and std
oneday['Helper_in_Cluster_exists'] = (oneday['Cluster_size']>oneday['Cluster_size_teams']).astype(int)
oneday['Gap_12_larger1'] = (oneday['Gap_Cluster12']>=60).astype(int)
oneday['Gap_23_larger1'] = (oneday['Gap_Cluster23']>=60).astype(int)
oneday['Cluster2_std_large'] = (oneday['Cluster2_std']>=oneday.Cluster2_std.mean()).astype(int)
oneday = oneday.drop_duplicates()

#captains only
oneday_c = oneday[(oneday["Teammates_front"]==0) & (oneday["Captain_in_Cluster"]==0)] 
oneday_c = oneday_c[oneday_c['Year'].astype(int) > 1980] #in 1980 we do not have any Scores

#captains only hyp
oneday_c_hyp = oneday[(oneday["Teammates_front_hyp"]==0) & (oneday["Captain_hyp_in_Cluster"]==0)] 
oneday_c_hyp = oneday_c_hyp[oneday_c_hyp['Year'].astype(int) > 1980] #in 1980 we do not have any Scores
oneday_c_hyp.loc[:, 'Cluster_size_teams_hyp'] = oneday_c_hyp.groupby(['Race_Year', 'Cluster'])['Rider'].transform('nunique')

#Find the winners for each group and include 'Cluster_size_winner' ... and same for cluster 2
winners = oneday_c_hyp[oneday_c_hyp['Place'] == 1][['Race_Year', 'Cluster_size_teams']]
oneday_c_hyp = pd.merge(oneday_c_hyp, winners, on='Race_Year', suffixes=('', '_winner'), how='left')
second = oneday_c_hyp[oneday_c_hyp['Cluster'] == 2][['Race_Year', 'Cluster_size_teams']]
oneday_c_hyp = pd.merge(oneday_c_hyp, second, on='Race_Year', suffixes=('', '_second'), how='left')
third = oneday_c_hyp[oneday_c_hyp['Cluster'] == 3][['Race_Year', 'Cluster_size_teams']]
oneday_c_hyp = pd.merge(oneday_c_hyp, third, on='Race_Year', suffixes=('', '_third'), how='left')

#Dummy for Gap Size and std
oneday_c_hyp['Gap_12_larger1'] = (oneday_c_hyp['Gap_Cluster12']>=60).astype(int)
oneday_c_hyp['Gap_23_larger1'] = (oneday_c_hyp['Gap_Cluster23']>=60).astype(int)
oneday_c_hyp = oneday_c_hyp.drop_duplicates()

#include 'Cluster_size_hyp_winner' ... and same for cluster 2
winners = oneday_c_hyp[oneday_c_hyp['Place'] == 1][['Race_Year', 'Cluster_size_teams_hyp']]
oneday_c_hyp = pd.merge(oneday_c_hyp, winners, on='Race_Year', suffixes=('', '_winner'), how='left')
second = oneday_c_hyp[oneday_c_hyp['Cluster'] == 2][['Race_Year', 'Cluster_size_teams_hyp']]
oneday_c_hyp = pd.merge(oneday_c_hyp, second, on='Race_Year', suffixes=('', '_second'), how='left')
third = oneday_c_hyp[oneday_c_hyp['Cluster'] == 3][['Race_Year', 'Cluster_size_teams_hyp']]
oneday_c_hyp = pd.merge(oneday_c_hyp, third, on='Race_Year', suffixes=('', '_third'), how='left')
oneday_c_hyp = oneday_c_hyp.drop_duplicates()

We downloaded a total of 298 one-day races.


### Summary statistics

In [147]:
# Table C.20: Summary Statistics Non-Dummies

# Create DataFrame 'races' for the calculations
races = pd.DataFrame()

# Calculate the mean for each race and each metric
races['Gap_Cluster12'] = oneday.groupby(['Race_Year'])['Gap_Cluster12'].mean()
races['Gap_Cluster23'] = oneday.groupby(['Race_Year'])['Gap_Cluster23'].mean()
races['Cluster_size_winner'] = oneday.groupby(['Race_Year'])['Cluster_size_winner'].mean()
races['Cluster_size_second'] = oneday.groupby(['Race_Year'])['Cluster_size_second'].mean()
races['Cluster_size_third'] = oneday.groupby(['Race_Year'])['Cluster_size_third'].mean()

# Combine the statistics into one DataFrame
summary_stats = pd.DataFrame({
    'Group 1 size': races['Cluster_size_winner'],
    'Group 2 size': races['Cluster_size_second'],
    'Group 3 size': races['Cluster_size_third'],
    'Gap between Groups 1 and 2': races['Gap_Cluster12'],
    'Gap between Groups 2 and 3': races['Gap_Cluster23']
})

# Use the .describe() function and filter for the relevant stats (mean, std, min, 50%, max)
summary_stats = summary_stats.describe().loc[['mean', 'std', 'min', '50%', 'max']]

# Rename index values to match your desired output
summary_stats.index = ['mean', 'std', 'min', '50%', 'max']

# Convert to LaTeX format and print
print(summary_stats.to_latex(index=True, float_format="%.2f"))

\begin{tabular}{lrrrrr}
\toprule
 & Group 1 size & Group 2 size & Group 3 size & Gap between Groups 1 and 2 & Gap between Groups 2 and 3 \\
\midrule
mean & 2.18 & 3.48 & 2.92 & 50.77 & 46.52 \\
std & 1.58 & 2.54 & 2.29 & 49.38 & 64.72 \\
min & 1.00 & 1.00 & 1.00 & 5.00 & 5.00 \\
50% & 2.00 & 3.00 & 2.00 & 28.00 & 23.00 \\
max & 10.00 & 12.00 & 11.00 & 219.00 & 408.00 \\
\bottomrule
\end{tabular}



In [148]:
# Table C.21: Mean occurrence of Dummies

cl_one = oneday[oneday['Cluster'] == 1]
cl_two = oneday[oneday['Cluster'] == 2]
cl_three = oneday[oneday['Cluster'] == 3]

# Calculate mean values for each group
group1_mean = cl_one[['Star', 'better_rider_in_Cluster', 'Helper_in_Cluster', 'Teammates_behind']].mean()
group2_mean = cl_two[['Star', 'better_rider_in_Cluster', 'Helper_in_Cluster', 'Teammates_behind']].mean()
group3_mean = cl_three[['Star', 'better_rider_in_Cluster', 'Helper_in_Cluster']].mean()

# Calculate overall mean (across all groups)
overall_mean = oneday[['Star', 'better_rider_in_Cluster', 'Helper_in_Cluster', 'Teammates_behind']].mean()

# Create DataFrame for the table
mean_occurrence = pd.DataFrame({
    'Group 1': group1_mean,
    'Group 2': group2_mean,
    'Group 3': group3_mean,
    'Overall': overall_mean
}).T

# Replace NaN values with 0.0 instead of an empty string (for compatibility)
mean_occurrence.fillna(0.0, inplace=True)

# Convert to LaTeX format
print(mean_occurrence[['Star', 'better_rider_in_Cluster', 'Helper_in_Cluster', 'Teammates_behind']].to_latex(index=True, float_format="%.3f"))



\begin{tabular}{lrrrr}
\toprule
 & Star & better_rider_in_Cluster & Helper_in_Cluster & Teammates_behind \\
\midrule
Group 1 & 0.323 & 0.288 & 0.038 & 0.165 \\
Group 2 & 0.227 & 0.454 & 0.072 & 0.126 \\
Group 3 & 0.138 & 0.325 & 0.057 & 0.000 \\
Overall & 0.221 & 0.368 & 0.059 & 0.149 \\
\bottomrule
\end{tabular}



### Regressions

In [149]:
# Table C.22: Linear Probability Model: Finishing in Group 1 - One-Day Races

# LHS
resultO12 = sm.ols(formula = 'Dummy_Cluster_1 ~   better_rider_around +Teammates_behind + Gap_12_larger1 + Gap_23_larger1 + Cluster_size_teams_winner + Cluster_size_teams_second +Cluster_size_teams_third+ Dummy_MSR + Dummy_LBL + Dummy_FW +Dummy_RVV + Dummy_PR + Dummy_SS ', data=oneday_c[(oneday_c['Cluster']==1) |(oneday_c['Cluster']==2)]).fit()
print(resultO12.summary())

# RHS
resultO123 = sm.ols(formula = 'Dummy_Cluster_1 ~   better_rider_around + Gap_12_larger1+ Gap_23_larger1 + Cluster_size_teams_winner + Cluster_size_teams_second +Cluster_size_teams_third + Dummy_MSR + Dummy_LBL + Dummy_FW +Dummy_RVV + Dummy_PR + Dummy_SS ', data=oneday_c[(oneday_c['Cluster']==1) |(oneday_c['Cluster']==2) |(oneday_c['Cluster']==3)]).fit()
print(resultO12.summary())


                            OLS Regression Results                            
Dep. Variable:        Dummy_Cluster_1   R-squared:                       0.265
Model:                            OLS   Adj. R-squared:                  0.249
Method:                 Least Squares   F-statistic:                     16.09
Date:                Tue, 15 Oct 2024   Prob (F-statistic):           1.52e-31
Time:                        17:46:21   Log-Likelihood:                -331.56
No. Observations:                 593   AIC:                             691.1
Df Residuals:                     579   BIC:                             752.5
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                                coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------
Intercept             

In [150]:
# Table C.23: Linear Probability Model: Winning the Race from Group 1 - One-Day Races

oneday_c_nsw= oneday_c[oneday_c['Cluster_size_teams_winner']!=1]
oneday_c_hyp_nsw= oneday_c_hyp[oneday_c_hyp['Cluster_size_winner']!=1]

#LHS
result1x_OD = sm.ols(formula = 'Win ~  better_rider_in_Cluster*Helper_in_Cluster + Teammates_behind + Gap_12_larger1 + Cluster_size_teams_winner + Cluster_size_teams_second+ Dummy_LBL + Dummy_FW +Dummy_RVV + Dummy_PR + Dummy_SS', data=oneday_c_nsw[oneday_c_nsw['Cluster']==1]).fit()#cov_type='cluster', cov_kwds={'groups': stage_c[stage_c['Cluster']==1]['Race']})
print(result1x_OD.summary())

#Middle column
result1_OD = sm.ols(formula = 'Win ~  better_rider_in_Cluster+Helper_in_Cluster + Teammates_behind + Gap_12_larger1 + Cluster_size_teams_winner + Cluster_size_teams_second+ Dummy_LBL + Dummy_FW +Dummy_RVV + Dummy_PR + Dummy_SS', data=oneday_c_nsw[oneday_c_nsw['Cluster']==1]).fit()#cov_type='cluster', cov_kwds={'groups': stage_c[stage_c['Cluster']==1]['Race']})
print(result1_OD.summary())

#RHS
# Winning the race for hypothetical teams
resultHyp_OD = sm.ols(formula = 'Win ~ better_rider_in_Cluster+Helper_hyp_in_Cluster + Teammates_behind_hyp + Gap_12_larger1 + Cluster_size_teams_hyp_winner + Cluster_size_teams_hyp_second+Dummy_LBL + Dummy_FW +Dummy_RVV + Dummy_PR + Dummy_SS', data=oneday_c_hyp_nsw[oneday_c_hyp_nsw['Cluster']==1]).fit()
print(resultHyp_OD.summary())


                            OLS Regression Results                            
Dep. Variable:                    Win   R-squared:                       0.138
Model:                            OLS   Adj. R-squared:                  0.081
Method:                 Least Squares   F-statistic:                     2.424
Date:                Tue, 15 Oct 2024   Prob (F-statistic):            0.00611
Time:                        17:46:22   Log-Likelihood:                -113.63
No. Observations:                 194   AIC:                             253.3
Df Residuals:                     181   BIC:                             295.7
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                                                coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------